# Custom Chatbot Project

### Dataset: character_descriptions.csv

This dataset contains character descriptions from theater, television, and film productions. This dataset contains 55 rows. Each row contains the name, description, medium, and setting. I will tranform the dataset to include all four columns to create a custom text column in a textual narrative form.

For example:

**text** = Name of the character is **Name**. **Description**. **Name** lives in **Setting**. **Name** usually likes to act in a **Medium**.

The **text** column from the transformed dataset will be used to build custom chatbot.

### Use case scenario

Quite often when designing a play for theater or for a movie, you would want to find a suitable character that matches the narrative. You would need to know where the character lives to match the culture, language accents and audience preferences. This custom chatbot makes it easier to find a character for a creative endeavour given a few traits.  

## Data Wrangling



In [10]:
!pip install openai==0.28
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.81.0
    Uninstalling openai-1.81.0:
      Successfully uninstalled openai-1.81.0


In [1]:
import pandas as pd
import numpy as np
import openai
import os
from openai.embeddings_utils import distances_from_embeddings

In [9]:
from google.colab import userdata
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = userdata.get('OPENAI_API_KEY')
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
MAX_TOKENS = 1000

In [3]:
df = pd.read_csv('/content/character_descriptions.csv')
df.describe()

,Name,Description,Medium,Setting
count,55,55,55,55
unique,55,55,7,6
top,Emily,"A young woman in her early 20s, Emily is an as...",Play,USA
freq,1,1,18,21


In [4]:
df['text'] = 'Name of the character is ' + df['Name'] + '. ' + df['Description'] + ' Lives in ' + df['Setting'] + '. ' + df['Name'] + ' usually likes to act in a ' + df['Medium'] + '.'
df["text"][0]

"Name of the character is Emily. A young woman in her early 20s, Emily is an aspiring actress and Alice's daughter. She has a bubbly personality and a quick wit, but struggles with self-doubt and insecurity. She's also in a relationship with George. Lives in England. Emily usually likes to act in a Play."

In [10]:
embeddings = []
for index, row in df.iterrows():
  response = openai.Embedding.create(
      input=row["text"],
      engine=EMBEDDING_MODEL_NAME
  )
  embeddings.extend([data["embedding"] for data in response["data"]])
df["embeddings"] = embeddings


In [11]:
df[["text", "embeddings"]].to_csv("character_descriptions_embeddings.csv")

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [12]:
df = pd.read_csv('/content/character_descriptions_embeddings.csv', index_col=0)
df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)
df.head()

,text,embeddings
0,Name of the character is Emily. A young woman ...,"[-0.016740478575229645, -0.011723474599421024,..."
1,Name of the character is Jack. A middle-aged m...,"[0.004325818736106157, -0.024151675403118134, ..."
2,Name of the character is Alice. A woman in her...,"[0.005864941515028477, -0.008803928270936012, ..."
3,Name of the character is Tom. A man in his 50s...,"[0.015749488025903702, -0.01712125912308693, -..."
4,Name of the character is Sarah. A woman in her...,"[-0.017749547958374023, -0.028192734345793724,..."


In [13]:
def question_embeddings(question):
  response = openai.Embedding.create(
      input=question,
      engine=EMBEDDING_MODEL_NAME
  )
  return response["data"][0]["embedding"]

In [14]:
q1 = "Who would be a good fit for a retiree role in an american sitcom?"
q2 = "Provide atleast two names of young female actors that can act in reality series?"
q1_embeddings = question_embeddings(q1)
q2_embeddings = question_embeddings(q2)

df['q1_distances'] = distances_from_embeddings(
  q1_embeddings,
  df['embeddings'].values,
  distance_metric="cosine"
)
df['q2_distances'] = distances_from_embeddings(
  q2_embeddings,
  df['embeddings'].values,
  distance_metric="cosine"
)
df.head()
dfq1 = df.sort_values(by=["q1_distances"], ascending=True)
dfq2 = df.sort_values(by=["q2_distances"], ascending=True)
dfq1.head()


,text,embeddings,q1_distances,q2_distances
7,Name of the character is John. A man in his 60...,"[0.01990000531077385, -0.01819981075823307, -0...",0.203889,0.282515
3,Name of the character is Tom. A man in his 50s...,"[0.015749488025903702, -0.01712125912308693, -...",0.207777,0.256660
33,Name of the character is Jake. A laid-back and...,"[-0.016017302870750427, -0.018636401742696762,...",0.214512,0.242575
52,Name of the character is Captain James. The ch...,"[-0.011542927473783493, -0.017732612788677216,...",0.216745,0.263883
54,Name of the character is Mr. Mercer. The bumbl...,"[-0.005541565362364054, -0.014116477221250534,...",0.216918,0.295276


In [15]:
dfq2.head()

,text,embeddings,q1_distances,q2_distances
0,Name of the character is Emily. A young woman ...,"[-0.016740478575229645, -0.011723474599421024,...",0.228769,0.207506
26,Name of the character is Olivia. A confident a...,"[-0.0067826067097485065, -0.014604642987251282...",0.237756,0.212682
32,Name of the character is Chloe. A driven and a...,"[-0.005866195075213909, -0.006107361055910587,...",0.235236,0.212810
30,Name of the character is Sophia. A fun-loving ...,"[0.02065388113260269, -0.013319969177246094, -...",0.250556,0.214061
14,Name of the character is Mia. A young Australi...,"[-0.010092192329466343, -0.016392482444643974,...",0.259387,0.216916


In [16]:
dfq1["text"].head(10).values

array(["Name of the character is John. A man in his 60s, John is a retired professor and Tom's father. He has a dry wit and a love of intellectual debate, but can also be stubborn and set in his ways. Lives in England. John usually likes to act in a Play.",
       "Name of the character is Tom. A man in his 50s, Tom is a retired soldier and John's son. He has a no-nonsense approach to life, but is haunted by his experiences in combat and struggles with PTSD. He's also in a relationship with Rachel. Lives in England. Tom usually likes to act in a Play.",
       "Name of the character is Jake. A laid-back and easygoing firefighter, Jake is the quintessential good guy. He's looking for someone who shares his values of honesty and integrity, and who is looking for a stable and committed relationship. He's a bit of a hopeless romantic, and is always looking for ways to sweep his partner off their feet. Lives in USA. Jake usually likes to act in a Reality Show.",
       "Name of the characte

In [19]:
import tiktoken
tokenizer = tiktoken.get_encoding("cl100k_base")
prompt_template = """
Answer the question based on the context below, and if the
question can't be answered based on the context, say
"I don't know"

Context:

{}

---

Question: {}
Answer:"""
def get_prompt(question, df):
  token_count = len(tokenizer.encode(prompt_template)) + len(tokenizer.encode(question))
  context_list = []
  for text in df["text"].head(10).values:
    token_count += len(tokenizer.encode(text))
    if token_count <= MAX_TOKENS:
        context_list.append(text)
    else:
        break
  prompt = prompt_template.format("\n\n###\n\n".join(context_list), question)
  return prompt

In [20]:
q1_prompt=get_prompt(q1, dfq1)
q2_prompt=get_prompt(q2, dfq2)

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"
response = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=q1_prompt,
    max_tokens=150
)
answer = response["choices"][0]["text"].strip()
print(answer)
basic_q1_response = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=q1,
    max_tokens=150
)
basic_q1_answer = basic_q1_response["choices"][0]["text"].strip()
print("Basic Answer for Q1:")
print(basic_q1_answer)

In [21]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"
response = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=q1_prompt,
    max_tokens=150
)
answer = response["choices"][0]["text"].strip()
print(answer)

basic_q1_response = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=q1,
    max_tokens=150
)
basic_q1_answer = basic_q1_response["choices"][0]["text"].strip()
print("Basic Answer for Q1:")
print(basic_q1_answer)

Mr. Mercer or Captain James would be a good fit for a retiree role in an American sitcom.
Basic Answer for Q1:
Betty White would be a good fit for a retiree role in an American sitcom. She has years of experience in the entertainment industry, is beloved by audiences of all ages, and has a natural charm and wit that would make her perfect for the role of a sassy and lovable retiree. Additionally, she has played similar roles in the past, such as in the sitcom "Hot in Cleveland," so she would bring a level of familiarity and expertise to the role.


### Question 2

In [22]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"
response = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=q2_prompt,
    max_tokens=150
)
answer = response["choices"][0]["text"].strip()
print(answer)

basic_q2_response = openai.Completion.create(
    model=COMPLETION_MODEL_NAME,
    prompt=q2,
    max_tokens=150
)
basic_q2_answer = basic_q2_response["choices"][0]["text"].strip()
print("Basic Answer for Q2:")
print(basic_q2_answer)

1. Maya
2. Olivia
Basic Answer for Q2:
1. Millie Bobby Brown
2. Zendaya
